In [76]:
import os
import sys
from pathlib import Path
from pprint import pprint
from datetime import datetime


from hydra import compose, initialize
from omegaconf import OmegaConf, DictConfig
import polars as pl
import polars.selectors as cs
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.ensemble import IsolationForest
from summarytools import dfSummary
from icecream import ic
import altair as alt

sys.path.append('/Users/zacklarsen/Documents/Projects/kaggle/kaggle-paris-housing-prices/src/')

from steps.ingest import ingest_raw_data
from steps.clean import clean_data

In [20]:
with initialize(version_base="1.3.2",
                config_path="../src/config",
                job_name="train"):
    cfg = compose(config_name="main")

In [21]:
pprint(cfg)

{'run': {'keep_columns': ['Income', 'Recency', 'NumWebVisitsMonth', 'Complain', 'age', 'total_purchases', 'enrollment_years', 'family_size'], 'remove_outliers_threshold': {'age': 90, 'income': 600000}}, 'target_var': 'hospital_death', 'train_test_split': {'random_state': 42, 'train_ratio': 0.75, 'validation_ratio': 0.125, 'test_ratio': 0.125}, 'numeric_features': ['pre_icu_los_days'], 'categorical_features': ['gender', 'ethnicity', 'hospital_admit_source', 'icu_admit_source'], 'base_path': '/Users/zacklarsen/Documents/Projects/kaggle', 'competition_path': '${base_path}/kaggle-paris-housing-prices', 'data_path': '${base_path}/data/paris_housing_prices', 'raw_data_path': '${data_path}/raw', 'processed_data_path': '${data_path}/processed', 'model_path': '${competition_path}/models', 'paths': {'mlflow': {'mlruns': '${competition_path}/mlruns'}, 'data': {'train_raw': '${raw_data_path}/train.csv', 'test_raw': '${raw_data_path}/test.csv', 'train_typed': '${processed_data_path}/train_typed.par

In [22]:
ic(cfg)

ic| cfg: {'run': {'keep_columns': ['Income', 'Recency', 'NumWebVisitsMonth', 'Complain', 'age', 'total_purchases', 'enrollment_years', 'family_size'], 'remove_outliers_threshold': {'age': 90, 'income': 600000}}, 'target_var': 'hospital_death', 'train_test_split': {'random_state': 42, 'train_ratio': 0.75, 'validation_ratio': 0.125, 'test_ratio': 0.125}, 'numeric_features': ['pre_icu_los_days'], 'categorical_features': ['gender', 'ethnicity', 'hospital_admit_source', 'icu_admit_source'], 'base_path': '/Users/zacklarsen/Documents/Projects/kaggle', 'competition_path': '${base_path}/kaggle-paris-housing-prices', 'data_path': '${base_path}/data/paris_housing_prices', 'raw_data_path': '${data_path}/raw', 'processed_data_path': '${data_path}/processed', 'model_path': '${competition_path}/models', 'paths': {'mlflow': {'mlruns': '${competition_path}/mlruns'}, 'data': {'train_raw': '${raw_data_path}/train.csv', 'test_raw': '${raw_data_path}/test.csv', 'train_typed': '${processed_data_path}/train_

{'run': {'keep_columns': ['Income', 'Recency', 'NumWebVisitsMonth', 'Complain', 'age', 'total_purchases', 'enrollment_years', 'family_size'], 'remove_outliers_threshold': {'age': 90, 'income': 600000}}, 'target_var': 'hospital_death', 'train_test_split': {'random_state': 42, 'train_ratio': 0.75, 'validation_ratio': 0.125, 'test_ratio': 0.125}, 'numeric_features': ['pre_icu_los_days'], 'categorical_features': ['gender', 'ethnicity', 'hospital_admit_source', 'icu_admit_source'], 'base_path': '/Users/zacklarsen/Documents/Projects/kaggle', 'competition_path': '${base_path}/kaggle-paris-housing-prices', 'data_path': '${base_path}/data/paris_housing_prices', 'raw_data_path': '${data_path}/raw', 'processed_data_path': '${data_path}/processed', 'model_path': '${competition_path}/models', 'paths': {'mlflow': {'mlruns': '${competition_path}/mlruns'}, 'data': {'train_raw': '${raw_data_path}/train.csv', 'test_raw': '${raw_data_path}/test.csv', 'train_typed': '${processed_data_path}/train_typed.par

In [23]:
cfg['paths']['data']['train_raw']

'/Users/zacklarsen/Documents/Projects/kaggle/data/paris_housing_prices/raw/train.csv'

## io

In [25]:
train = pl.read_csv(cfg.paths.data.train_raw)

In [34]:
train.glimpse()

Rows: 22730
Columns: 18
$ id                <i64> 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
$ squareMeters      <i64> 34291, 95145, 92661, 97184, 61752, 30300, 19341, 58110, 20537, 89396
$ numberOfRooms     <i64> 24, 60, 45, 99, 100, 36, 95, 3, 84, 35
$ hasYard           <i64> 1, 0, 1, 0, 0, 0, 1, 1, 0, 1
$ hasPool           <i64> 0, 1, 1, 0, 0, 0, 0, 0, 1, 1
$ floors            <i64> 47, 60, 62, 59, 57, 35, 37, 1, 44, 96
$ cityCode          <i64> 35693, 34773, 45457, 15113, 64245, 19069, 23939, 67658, 9560, 33928
$ cityPartRange     <i64> 2, 1, 4, 1, 8, 8, 9, 8, 3, 6
$ numPrevOwners     <i64> 1, 4, 8, 1, 4, 4, 6, 4, 10, 7
$ made              <i64> 2000, 2000, 2020, 2000, 2018, 1998, 2017, 2003, 1994, 1993
$ isNewBuilt        <i64> 0, 0, 1, 0, 1, 1, 0, 1, 0, 0
$ hasStormProtector <i64> 1, 1, 1, 1, 0, 0, 0, 0, 0, 0
$ basement          <i64> 8, 729, 7473, 6424, 7151, 8181, 7173, 3164, 5026, 7936
$ attic             <i64> 5196, 4496, 8953, 8522, 2786, 1826, 9233, 3389, 7669, 2956
$ garage            <

In [33]:
train.describe()

describe,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0,22730.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",11364.5,46586.218302,48.241091,0.475891,0.45275,47.305983,50013.795996,5.585042,5.620766,2008.005059,0.467972,0.460009,5283.646634,5067.981698,530.469644,0.461681,5.153058,4.6345e6
"""std""",6561.730145,49521.244701,28.226428,0.499429,0.497773,47.777207,30006.637729,2.739533,2.713026,118.826777,0.498984,0.498409,3047.084412,3097.347939,274.840604,0.49854,3.055246,2.9252e6
"""min""",0.0,89.0,1.0,0.0,0.0,1.0,3.0,1.0,1.0,1990.0,0.0,0.0,4.0,1.0,4.0,0.0,0.0,10313.5
"""25%""",5682.0,20392.0,25.0,0.0,0.0,25.0,22936.0,3.0,3.0,2000.0,0.0,0.0,2977.0,2599.0,297.0,0.0,3.0,2041739.1
"""50%""",11365.0,44484.0,47.0,0.0,0.0,45.0,50414.0,6.0,6.0,2006.0,0.0,0.0,5359.0,4977.0,515.0,0.0,5.0,4450823.4
"""75%""",17047.0,71549.0,75.0,1.0,1.0,69.0,76291.0,8.0,8.0,2014.0,1.0,1.0,7746.0,7652.0,767.0,1.0,8.0,7159919.5
"""max""",22729.0,6.07133e6,100.0,1.0,1.0,6000.0,491100.0,10.0,10.0,10000.0,1.0,1.0,91992.0,96381.0,9017.0,1.0,10.0,1.0004e7


## Year to age

In [92]:
made_counts = (
    train
    .select(pl.col(["made"]).value_counts())
    .unnest(columns="made")
    .sort(by="counts", descending=True)
)

In [93]:
made_counts

made,counts
i64,u32
2000,3588
2003,1156
2014,1070
2015,1036
2007,1021
2008,981
2009,972
2019,956
2013,934


In [87]:
alt.Chart(made_counts).mark_bar().encode(
    alt.X("made:Q", bin=True),
    y='counts',
)

alt.Chart(...)

In [94]:
current_year = datetime.now().year

(
    train
    .with_columns(age = current_year - pl.col("made"))
    .select(pl.col(["made", "age"]))
    .limit(5)
)

made,age
i64,i64
2000,23
2000,23
2020,3
2000,23
2018,5


## Outliers

In [74]:
outliers_removed = (
    train
    .with_columns(pl.col("basement").map_batches(lambda x: np.abs(stats.zscore(x))).alias("basement_z_score_abs"))
    .with_columns(pl.col("squareMeters").map_batches(lambda x: np.abs(stats.zscore(x))).alias("square_meters_z_score_abs"))
    .with_columns(pl.col("floors").map_batches(lambda x: np.abs(stats.zscore(x))).alias("floors_z_score_abs"))
    .with_columns(pl.col("attic").map_batches(lambda x: np.abs(stats.zscore(x))).alias("attic_z_score_abs"))
    .with_columns(pl.col("garage").map_batches(lambda x: np.abs(stats.zscore(x))).alias("garage_z_score_abs"))
    .filter(pl.col("basement_z_score_abs") < 3)
    .filter(pl.col("square_meters_z_score_abs") < 3)
    .filter(pl.col("floors_z_score_abs") < 3)
    .filter(pl.col("attic_z_score_abs") < 3)
    .filter(pl.col("garage_z_score_abs") < 3)
).to_pandas()

In [145]:
outliers_removed = (
    train
    .with_columns(
        pl.col(["basement", "squareMeters", "floors", "attic", "garage"])
        .map_batches(lambda x: np.abs(stats.zscore(x)))
        .name.suffix("_z_score_abs")
    )
    .filter(
        pl.fold(
            acc=pl.lit(True),
            function=lambda acc, x: acc & x,
            exprs=cs.contains("_z_score") < 3,
        )
    )
)

outliers_removed_cols = outliers_removed.columns

outliers_removed_cols.sort()

In [146]:
outliers_removed.select(outliers_removed_cols).limit(5)

attic,attic_z_score_abs,basement,basement_z_score_abs,cityCode,cityPartRange,floors,floors_z_score_abs,garage,garage_z_score_abs,hasGuestRoom,hasPool,hasStorageRoom,hasStormProtector,hasYard,id,isNewBuilt,made,numPrevOwners,numberOfRooms,price,squareMeters,squareMeters_z_score_abs
i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,f64
5196,0.041332,8,1.731413,35693,2,47,0.006405,369,0.587516,3,0,0,1,1,0,0,2000,1,24,3436795.2,34291,0.248287
4496,0.184672,729,1.494789,34773,1,60,0.265698,277,0.922263,6,1,0,1,0,1,0,2000,4,60,9.519958e6,95145,0.980586
8953,1.254332,7473,0.718523,45457,4,62,0.30756,245,1.038696,9,1,1,1,1,2,1,2020,8,45,9276448.1,92661,0.930425
8522,1.115178,6424,0.374252,15113,1,59,0.244767,256,0.998672,9,0,1,1,0,3,0,2000,1,99,9725732.2,97184,1.021761
2786,0.73677,7151,0.612846,64245,8,57,0.202905,863,1.209929,7,0,0,0,0,4,1,2018,4,100,6181908.8,61752,0.306255


In [30]:
train_pd = train.to_pandas()

In [31]:
train_pd

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22725,22725,55825,84,1,0,70,12031,3,10,2000,0,0,4477,786,345,0,0,5594137.1
22726,22726,65870,88,1,0,49,23197,9,9,2015,0,1,4811,2454,755,0,7,6594705.0
22727,22727,93192,42,1,0,39,8539,10,5,2014,1,0,5595,4072,789,0,0,9321511.4
22728,22728,65797,86,1,0,89,23197,2,10,2000,1,0,5358,2513,411,0,0,6584708.2


## dfsummary

In [27]:
dfSummary(train_pd)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,id[int64],Mean (sd) : 11364.5 (6561.7)min < med < max:0.0 < 11364.5 < 22729.0IQR (CV) : 11364.5 (1.7),"22,730 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy89olMNAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABxklEQVR4nO3XQU7CYBBA4fkbJSCkDSHCAbwBSw7hYT0EJ2DhxhU7EtKUUDEarCcAKTXqS963baczixcSUtM0If132V8fIF3i5rsXUkqDiOhd+f0sIj6vnO06T91Nvbvr/HvTNIdTD8+GmlIaTCaTxzzPx223Ho/H27quH0aj0UuWZR+/OU/dTb37J+Z3u12ZUno6Fet3v6i9PM/Hi8XiUBTFW5vF6/V6vFqt7ufz+fNsNivbzHadp+6m3t11vqqq/nK5HG+3215EXBVqREQURfE2nU5f2ywvy3IQETEcDlvPdp2n7qbe/RPzETE499A/U0IwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEKwVCFYKhCMFQhGKoQDFUIhioEQxWCoQrBUIVgqEIwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEKwVCFYKhCMFQhGKoQDFUIhioEQxWCoQrBUIVgqEIwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEK4eaSl6qq6rf98H6/70dE1HXd32w2d785T91Nvbvr/CV9fQGSljnmc7n3zQAAAABJRU5ErkJggg=="">",0(0.0%)
2,squareMeters[int64],Mean (sd) : 46586.2 (49521.2)min < med < max:89.0 < 44484.0 < 6071330.0IQR (CV) : 51154.2 (0.9),"7,319 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy89olMNAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABaUlEQVR4nO3XT2oCMRiH4aSI6GYGETyMh+hhewhvIyUuqi1Iuuum1D/jov3R59km+cjiZYbU3nuBv+7pty8At5hd21BrXZZS5hPnf/TejxPPwpeLodZal+v1+nkYhtWU4YfD4bXW+iJWHnXtizofhmG13W6P4zie7hncWlvsdrvVfr+fl1KEykOu/vpLKWUcx9Nms3mbMH854Qx84zFFBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEmN2yqbW2uHdwa21xPp9npZSh1nr3xfh33nvvp58WPwF79y3yrjldnwAAAABJRU5ErkJggg=="">",0(0.0%)
3,numberOfRooms[int64],Mean (sd) : 48.2 (28.2)min < med < max:1.0 < 47.0 < 100.0IQR (CV) : 50.0 (1.7),100 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy89olMNAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACo0lEQVR4nO3cTW7aQBiA4W9KQAZUW4gKdr5ApS5YsukNethuu+sCLsAN2CAkZIxwg/jtrqJq0sCMJ5lv8j77MY71auQZ2zGXy0WA0H146xMAbvHw1ieAcBhj2iLScjjE/nK5PNZ1PteiDTXkix4iY0y73+9/S9O0Z3uMzWZTGGO++7huUYZax0Vfr9dbY8wPEdlZDNcYeStN0954PH7Msuzuv7ksy2QymfRWq1VLRAj1Rk4XfbFYfJxOp1/zPP/UaDTuHu9zZvEty7LdYDD4ZTm8XevJXIk1VBGxv+hFUbSTJOmMRqPdcDgs7hnre2Z5r6IO1VW327WdXbzNLO/Vi6E6Lko03qshQP8N1XVRovleDWF5aUa1XpTUca/mMJun5/O5afObCNNN96gOK0HrezWX2fxwOCSn0+nzbrf7KSK2K1gEJOTFlPVsPp/Pe7PZrHM8Hhu+Tg6vK+RQRcRuNi+KglV3ZIIPVaPT6dQUkdQYYzOcnZInEGrNqqpqHg6HL3meN97bUy2fCLVm+/3+gada9SNUT3iqVS9enIYKXmdUx0UFm/b4w1uorosKNu1xzVuoLosKETbt8TfviynbRQWb9rjGYgoqECpUYB81MrG+GkmoEYn51UhCjUu0r0YSaoRifDWSUAPD07ynEWpAeJr3PEINCE/znkeoAeJp3r/Y8IcKzKiojc9vxQgVtfD9rRihoha+vxUjVNTK17diLKagAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqHDT/0ctyzK598Db7TYREamqKlkul53XHK/1t7Wet+v4W/r6DQWCrQlCsOXTAAAAAElFTkSuQmCC"">",0(0.0%)
4,hasYard[int64],1. 02. 1,"11,913 (52.4%)10,817 (47.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy89olMNAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABOklEQVR4nO3bQWrDMBRF0a8iQjORMAYvxovoYrsI7cYYa9CETNRB50FU5iki96zgYy6GBD9XSjFAwZnZ1cwuvQ8Z3KOUcut9xKvz8zx/hRCm3oeMLOe8O+e+Ce45H0KY1nW9xRjvvY8Z0XEcnymladu2i5kR2xPezCzGeF+W5af3MQO79j5gBB+9D8D7IDbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQcab/X0A2PuQUfHs6vmc855SmowPAP8t57yb2aP3Ha+Owcs5GLxUcEz5oMKbrR5vr0ZM+Sox12vHlK8Cc71zMOWrx6/1RvypCxligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkmPJV4PmcgylfJeZ67Ri81GPw0ugXCltq3zyAhfwAAAAASUVORK5CYII="">",0(0.0%)
5,hasPool[int64],1. 02. 1,"12,439 (54.7%)10,291 (45.3%)","<img src = ""data:image/png;base64